# Mean Teacher Bert Assessment



# Requirement Installation 

In [2]:
#installing requirements
!pip install tensorflow_text
!pip install textattack
!pip install sentence_transformers

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 373 kB 5.2 MB/s 
     |████████████████████████████████| 769 kB 43.4 MB/s 
     |████████████████████████████████| 69 kB 7.5 MB/s 
     |████████████████████████████████| 322 kB 64.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 298 kB 61.2 MB/s 
     |████████████████████████████████| 60 kB 7.5 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.2 MB/s 
     |████████████████████████████████| 895 kB 53.3 MB/s 
     |████████████████████████████████| 61 kB 529 kB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
     |████████████████████████████████| 298 kB 58.2 MB/s 
     |████████████████████████████████| 290 kB 55.5 MB/s 
     |████████████████████████████████| 290 kB 48.6 MB/s 
     |█████████████████


# Importing and Declaration

In [3]:
import os 
os.chdir('/content/drive/MyDrive/Master Thesis/Mean-Teacher-BERT-Robustness-Assessment/')

In [4]:
# importing libraries 
import os 
import datetime

import pandas as pd 
import numpy as np
import tensorflow as tf

from ipywidgets import *
from IPython.display import display


from Utils.utils import label_to_int, dataset_split, convert_to_category, create_tokenizer, data_tokenization
from Utils.data_loader import data_loader
from Bert.bert_model import bert_model



from MeanTeacher.train_mean_teacher import train_mean_teacher

from sklearn.metrics import classification_report

import random 
random.seed(42)

## Data Related Questionaire

In [5]:
data_name=widgets.Dropdown(options=['codalab','IMDB' ],value='IMDB',description='DATA:',disabled=False,) 
pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,)
widgets.HBox([data_name, pretrained_weights])

### Data specific detail

In [6]:
'''
NOT required now as previous cell does the job. So disabling boxes
'''
# Details related to the dataset
# columns names 
feature_col=widgets.Text(value='tweet',placeholder='Enter Feature Col Name',description='Feature Column:',disabled=False)
target_col=widgets.Text(value='label',placeholder='Enter Target Col Name',description='Target Column:',disabled=False) 
aug_col=widgets.Text(value='aug_tweet',placeholder='Enter Augmented data Col Name',description='Augment Column:',disabled=False)   

#pretrained model details
 # widgets.HBox([feature_col,target_col,aug_col,pretrained_weights])

In [7]:
#details of the class
class_1=widgets.Text(value='real',placeholder='Enter class name 1',description='class 1:',disabled=False)
class_2=widgets.Text(value='fake',placeholder='Enter class name 2',description='class 2:',disabled=False) 
tf.print("Class 1 label would be 0. \n")
# widgets.HBox([class_1,class_2])

Class 1 label would be 0. 



In [8]:
# #Data location 
Data_loc=widgets.Text(value='Data/codalab/',placeholder='Enter Data location ',description=' Data Location Column:',disabled=False)   
# widgets.HBox([Data_loc])

In [9]:
'''
Adjusting feature as per data 
'''

if data_name.value=='codalab':
    feature_col.value='tweet'
    target_col.value='label'
    aug_col.value='aug_tweet'
    class_1.value='real'
    class_2.value='fake'
    Data_loc.value= 'Data/codalab/'
elif data_name.value=='IMDB':
    feature_col.value='review'
    target_col.value='sentiment'
    aug_col.value='aug_tweet'
    class_1.value='positive'
    class_2.value='negative'
    Data_loc.value= 'Data/IMDB/'



## Hyperparameter selections

In [10]:
# Parameter declaration 
max_len=widgets.BoundedIntText(value=100,min=10,max=512,step=10,description='Max Length:',disabled=False,)

learning_rate= widgets.Dropdown(options=[2e-5, 1e-5, 1e-3],value=2e-5,description='Learning Rate:',disabled=False,)

epochs= widgets.Dropdown(options=[3, 4, 5,6],value=3,description='Epochs:',disabled=False,)

batch_size= widgets.Dropdown(options=[4, 5,6,7,8],value=4,description='Batch Size:',disabled=False,)

#for mean teacher model
alpha= widgets.BoundedFloatText(value=0.999,min=0.90,max=0.999,step=0.0001,description='Alpha:',disabled=False, readout_format='.3f')
ratio= widgets.BoundedFloatText(value=0.5,min=0.1,max=0.9,step=0.1,description='Ratio:',disabled=False, readout_format='.2f')
# pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,)

tf.print('\n Max Length: Length of the article')
display(max_len)
tf.print('\n Learning rate: Learning rate for training the model')
display(learning_rate)
tf.print('\n Epochs')
display(epochs)
tf.print('\n batch_size')
display(batch_size)
tf.print('\n Alpha: Parameter related to Mean teacher model, its the proportion of student weight that teacher model will get each step.')
display(alpha)
tf.print("\n Ratio: Parameter related to Mean Teacher model, that manage the proportion of classification cost and consistency cost in calculating overall cost.")
display(ratio)

# widgets.HBox([max_len, learning_rate, epochs, batch_size,alpha,ratio])


 Max Length: Length of the article


BoundedIntText(value=100, description='Max Length:', max=512, min=10, step=10)


 Learning rate: Learning rate for training the model


Dropdown(description='Learning Rate:', options=(2e-05, 1e-05, 0.001), value=2e-05)


 Epochs


...

Dropdown(description='Batch Size:', options=(4, 5, 6, 7, 8), value=4)


 Alpha: Parameter related to Mean teacher model, its the proportion of student weight that teacher model will get each step.


BoundedFloatText(value=0.999, description='Alpha:', max=0.999, min=0.9, step=0.0001)


 Ratio: Parameter related to Mean Teacher model, that manage the proportion of classification cost and consistency cost in calculating overall cost.


BoundedFloatText(value=0.5, description='Ratio:', max=0.9, min=0.1, step=0.1)

# Reading dataset

In [11]:
df_train,df_test,df_aug_unlabel= data_loader(Data_loc.value,target_col.value,aug_col.value,class_1.value)

In [12]:
len(df_train),len(df_test)

(6050, 2000)

In [13]:
tf.print(df_train.groupby(target_col.value).count())
tf.print(df_test.groupby(target_col.value).count())

           review
sentiment        
0            3025
1            3025
           review
sentiment        
0            1000
1            1000


# Training 

<font color='blue'>This section is to train and save the model. <br>
__If model is already trained and saved, directly move forward for attack and evaluation section.__ </font>

## BERT Model Training (Baseline)

In [ ]:
pretrained_weights.value

'distilbert-base-uncased'

In [ ]:
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:
#printing hyperparameters set for training 
print(f'Alpha: {alpha.value}, Max len : {max_len.value}, Learning Rate: {learning_rate.value}, Batch Size: {batch_size.value}, Ratio: {ratio.value}, Epochs: {epochs.value}')

Alpha: 0.999, Max len : 100, Learning Rate: 2e-05, Batch Size: 4, Ratio: 0.5, Epochs: 3


In [ ]:
bert=bert_model(pretrained_weights.value,max_len.value,learning_rate.value)
bert.summary()

Model selected: distilbert-base-uncased


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 100, 768),                                                   
                                 hidden_states=None                                           

In [ ]:
print(datetime.datetime.now())
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size.value,epochs=epochs.value,verbose=1)

2021-11-27 10:53:40.537232
Epoch 1/3


KeyboardInterrupt: ignored

In [ ]:
# evaluating model 
pred_label=bert.predict(x_test)

print(classification_report(np.argmax(y_test,1),np.argmax(pred_label,1)))

In [ ]:
#saving the model 
if  pretrained_weights.value == 'distilbert-base-uncased':
    bert.save_weights('SavedModels/'+data_name.value+'/DistilBERT_model.h5')
else:
    bert.save_weights('SavedModels/'+data_name.value+'/BERT_model.h5')

## Adversarial Mean Teacher BERT Training(Proposed)

In [ ]:
# Training
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)
x_unlabel,_,_=data_tokenization(df_aug_unlabel,aug_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:
#printing hyperparameters set for training 
print(f'Alpha: {alpha.value}, Max len : {max_len.value}, Learning Rate: {learning_rate.value}, Batch Size: {batch_size.value}, Ratio: {ratio.value}, Epochs: {epochs.value}')


In [ ]:
print(datetime.datetime.now())
# training the model
with tf.device('/GPU:0'):
    student, teacher= train_mean_teacher(x_train, y_train, x_unlabel, 
                                         pretrained_weights.value,
                                         epochs.value,batch_size.value,
                                         learning_rate.value,max_len.value,alpha.value)

In [ ]:
if  pretrained_weights.value == 'distilbert-base-uncased':
    teacher.save_weights('SavedModels/'+data_name.value+'/DistilBERT_teacher.h5')
else:
    teacher.save_weights('SavedModels/'+data_name.value+'/BERT_teacher.h5')


In [ ]:
#evaluating test data 
pred_label=teacher.predict(x_test)

In [ ]:
print(classification_report(np.argmax(y_test,1),np.argmax(pred_label,1)))

# Attack And Evaluation

In [14]:
import textattack
from Attack_model.TextAttack import attack_model

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 39.3MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp_m4tr4ry.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [15]:
# bert_to_attack=widgets.Textarea(value='SavedModels/Codalab/BERT_model.h5',placeholder='Enter Model location ',description=' BERT Model location:',disabled=False)
# MTbert_to_attack=widgets.Textarea(value='SavedModels/Codalab/BERT_teacher.h5',placeholder='Enter Model location ',description=' MTBERT Model location:',disabled=False)   
# pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,) 
model_name=widgets.Dropdown(options=['BERT','DistilBERT']
                               ,value='BERT',
                               description='Model Name:',
                               disabled=False,) 
tf.print('\n model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter')
display(model_name)

# tf.print('\n Please select the location of BERT and MT BERT model.\n')
# display(bert_to_attack, MTbert_to_attack)
# tf.print('\n Please select the pretrained model weights.')
# display(pretrained_weights)



 model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter


Dropdown(description='Model Name:', options=('BERT', 'DistilBERT'), value='BERT')

In [16]:
if model_name.value=='BERT':
  bert_to_attack= 'SavedModels/'+data_name.value+'/BERT_model.h5'
  MTbert_to_attack='SavedModels/'+data_name.value+'/BERT_teacher.h5'
  pretrained_weights.value = 'bert-base-uncased'
else:
  bert_to_attack= 'SavedModels/'+data_name.value+'/DistilBERT_model.h5'
  MTbert_to_attack='SavedModels/'+data_name.value+'/DistilBERT_teacher.h5'
  pretrained_weights.value = 'distilbert-base-uncased'
print(model_name.value), bert_to_attack

BERT


(None, 'SavedModels/IMDB/BERT_model.h5')

In [17]:
# attack parameters 

# Query Budget is maximum query allowed
query_budget= widgets.BoundedIntText(value=6000,min=10,max=10000,step=20,description='Query Budget:',disabled=False,)

# number of example to take for attack from test dataset randomly 
num_examples=widgets.BoundedIntText(value=2000,min=10,max=len(df_test),step=20,description='Number of Example:',disabled=False,)

# attack recipes 
attack_recipe=widgets.Dropdown(options=['TextFoolerJin2019',
                                        'PWWSRen2019',
                                        'BAEGarg2019',
                                        'TextBuggerLi2018']
                               ,value='TextFoolerJin2019',
                               description='Attack Recipe:',
                               disabled=False,) 


tf.print('\n query_budget: Maximum Query allowed to attack model')
display(query_budget)
tf.print('\n num_examples: Number of example to take for attack from test dataset randomly. ')
display(num_examples)
# tf.print('\n attack_recipe: Attack methodolgy to attack the model. ')
# display(attack_recipe)



 query_budget: Maximum Query allowed to attack model


BoundedIntText(value=6000, description='Query Budget:', max=10000, min=10, step=20)


 num_examples: Number of example to take for attack from test dataset randomly. 


BoundedIntText(value=2000, description='Number of Example:', max=2000, min=10, step=20)

In [18]:
'''
Some of the attack recipe provided by textattack is not working with lengthy article. So, 
I am filtering the with the length. May be in future I will remove it.
'''
if not os.path.exists('Data/'+data_name.value+'/AdversarialTestData/test_adv_'+str(num_examples.value)+'.csv'):
    df_test['len']=df_test.apply(lambda row: len(row[feature_col.value].split()),axis=1)
    print(f'Average length of articles are: {df_test["len"].mean()}')
    # df_test_for_attack= df_test[(df_test['len']>0) & (df_test['len']<100)][[feature_col.value,target_col.value]].dropna().reset_index(drop=True)
    df_test_for_attack=df_test[[feature_col.value,target_col.value]].sample(frac=1).reset_index(drop=True)
    df_test_for_attack= df_test_for_attack[:num_examples.value+1]
    # saving the csv file in AdversarialTestData folder, in order to correctly experiment all recipes with same data
    df_test_for_attack.to_csv('Data/'+data_name.value+'/AdversarialTestData/test_adv_'+str(num_examples.value)+'.csv',index=False)
else:
    tf.print('Reading the saved file')
    df_test_for_attack= pd.read_csv('Data/'+data_name.value+'/AdversarialTestData/test_adv_'+str(num_examples.value)+'.csv')

print('Length of dataset', len(df_test_for_attack))
df_test_for_attack.head()

Reading the saved file
Length of dataset 2000


,review,sentiment
0,our reviewer from toronto told you what you ne...,0
1,i have recently seen this movie due to jake s ...,0
2,if you enjoy sitting in the dark both literal...,1
3,i m sitting around going through movie listing...,0
4,from the nepotism capitol of the world comes a...,1


In [19]:
df_test_for_attack.groupby(target_col.value).count()

,review
sentiment,
0,1000
1,1000


In [20]:

#creating dataset for the attacking
dataset_for_attack=list(df_test_for_attack.itertuples(index=False, name=None))
dataset_for_attack = textattack.datasets.Dataset(dataset_for_attack)

In [21]:
# Loading the model
bert= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert.load_weights(bert_to_attack)

bert_teacher= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert_teacher.load_weights(MTbert_to_attack)

Model selected: bert-base-uncased


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model selected: bert-base-uncased


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### TextFooler
__TextFoolerJin2019 - Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment.__<br>
In this paper, we present TextFooler, a simple but strong baseline to generate natural adversarial text. By applying it to two fundamental natural language tasks, text classification and textual entailment, we successfully attacked three target models, including the powerful pre-trained BERT, and the widely used convolutional and recurrent neural networks. We demonstrate the advantages of this framework in three ways: (1) effective---it outperforms state-of-the-art attacks in terms of success rate and perturbation rate, (2) utility-preserving---it preserves semantic content and grammaticality, and remains correctly classified by humans, and (3) efficient---it generates adversarial text with computational complexity linear to the text length. *The code, pre-trained target models, and test examples are available at https://github.com/jind11/TextFooler.


In [ ]:
attack_recipe.value= 'TextFoolerJin2019'
tf.print(f'****Attack Recipe:{attack_recipe.value}*****')
tf.print(f'****Query size:{query_budget.value}*****')
tf.print(f'****Number of examples:{num_examples.value}*****')
tf.print(f'****Attacking model :{model_name.value}*****')

****Attack Recipe:TextFoolerJin2019*****
****Query size:6000*****
****Number of examples:2000*****
****Attacking model :BERT*****


#### BERT

In [ ]:
attack_model(bert, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

#### MT BERT

In [ ]:
attack_model(bert_teacher, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

### TextBugger
__TextBugger: Generating Adversarial Text Against Real-world Applications.__ <br>
Deep Learning-based Text Understanding (DLTU) is the backbone technique behind various applications, including question answering, machine translation, and text classification. Despite its tremendous popularity, the security vulnerabilities of DLTU are still largely unknown, which is highly concerning given its increasing use in security-sensitive applications such as sentiment analysis and toxic content detection. In this paper, we show that DLTU is inherently vulnerable to adversarial text attacks, in which maliciously crafted texts trigger target DLTU systems and services to misbehave. Specifically, we present TextBugger, a general attack framework for generating adversarial texts. In contrast to prior works, TextBugger differs in significant ways: (i) effective -- it outperforms state-of-the-art attacks in terms of attack success rate; (ii) evasive -- it preserves the utility of benign text, with 94.9\% of the adversarial text correctly recognized by human readers; and (iii) efficient -- it generates adversarial text with computational complexity sub-linear to the text length. We empirically evaluate TextBugger on a set of real-world DLTU systems and services used for sentiment analysis and toxic content detection, demonstrating its effectiveness, evasiveness, and efficiency. For instance, TextBugger achieves 100\% success rate on the IMDB dataset based on Amazon AWS Comprehend within 4.61 seconds and preserves 97\% semantic similarity. We further discuss possible defense mechanisms to mitigate such attack and the adversary's potential countermeasures, which leads to promising directions for further research.



In [ ]:
attack_recipe.value= 'TextBuggerLi2018'
tf.print(f'****Attack Recipe:{attack_recipe.value}*****')
tf.print(f'****Query size:{query_budget.value}*****')
tf.print(f'****Number of examples:{num_examples.value}*****')

****Attack Recipe:TextBuggerLi2018*****
****Query size:6000*****
****Number of examples:2000*****


#### BERT

In [ ]:
attack_model(bert, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

#### MT BERT

In [ ]:
attack_model(bert_teacher, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

### PWWS
__PWWSRen 2019 - Generating Natural Language Adversarial Examples through Probability Weighted Word Saliency. Shuhuai Ren, Yihe Deng, Kun He, Wanxiang Che__<br>
Based on the synonyms substitution strategy, they introduced a new word replacement order determined by both the word saliency and the classification probability, and propose a greedy algorithm called probability weighted word saliency (PWWS) for text adversarial attack.

In [ ]:
attack_recipe.value= 'PWWSRen2019'

# because of issue in PWWS attack provided by textattack I need to change the number of  queries = number of examples
# query_budget.value = 6000
tf.print(f'****Attack Recipe:{attack_recipe.value}*****')
tf.print(f'****Query size:{query_budget.value}*****')
tf.print(f'****Number of examples:{num_examples.value}*****')

****Attack Recipe:PWWSRen2019*****
****Query size:6000*****
****Number of examples:2000*****


#### BERT

In [ ]:
attack_model(bert, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

#### MT BERT

In [ ]:
attack_model(bert_teacher, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Output hidden; open in https://colab.research.google.com to view.

### BAE
__BAEGarg2019-BAE (BAE: BERT-Based Adversarial Examples)__ <br>
BAE, a black box attack for generating adversarial examples using contextual perturbations from a BERT masked language model. BAE replaces and inserts tokens in the original text by masking a portion of the text and leveraging the BERT-MLM to generate alternatives for the masked tokens. Through automatic and human evaluations, we show that BAE performs a stronger attack, in addition to generating adversarial examples with improved grammaticality and semantic coherence as compared to prior work.


In [22]:
attack_recipe.value= 'BAEGarg2019'
tf.print(f'****Attack Recipe:{attack_recipe.value}*****')
tf.print(f'****Query size:{query_budget.value}*****')
tf.print(f'****Number of examples:{num_examples.value}*****')

****Attack Recipe:BAEGarg2019*****
****Query size:6000*****
****Number of examples:2000*****


#### BERT

In [ ]:
attack_model(bert, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/IMDB/BAEGarg2019/BERT_nexp2000_qb6000_2021-12-12_14:48.csv
  0%|          | 0/2000 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/2000 [00:00<09:04,  3.67it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (54%)]] --> [[[SKIPPED]]]

our reviewer from toronto told you what you need to know about this film  except note that it needs editing the hand held technique gets really old  really fast  i saw this film last night in menerbes  france we are in the luberon valley  which is covered with vineyards and of course wine makers  they were all there in the salle de polyvalente for the showing crammed in  polite  patient  genial  although my french is testy  i got the gist of the film but noted that the audience loved the  old  terror growers interviewed esp  the one from a communist village in languedoc  he got a lot of laughs  this is unusual in france laughing aloud  there is no question which side of the terror globalization war they are on  sm




Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   0%|          | 2/2000 [00:21<5:51:45, 10.56s/it]

...

--------------------------------------------- Result 143 ---------------------------------------------
[[0 (73%)]] --> [[[FAILED]]]

 toi le venin  is robert hossein s masterpiece and one of the great thrillers of the fifties based on a frederic dard novel a writer the director often worked with  see also  le monte charge  which hossein did not direct but in which he was the lead too the screenplay grabs you from the first pictures on a desert road by night where a beautiful blonde might be the fieriest of the criminals to the mysterious house where he finds his femme fatale  and her sister then begins a cat and mouse play  one of the sisters is in a wheelchair  but is she really disabled which one is the criminal who tried to kill the hero on that night       the two actresses marina vlady and the late odile versois were sisters     turn off all the lights before watching highly suspenseful 




[Succeeded / Failed / Skipped / Total] 79 / 57 / 8 / 144:   7%|▋         | 144/2000 [22:38<4:51:48,  9.43s/it]

--------------------------------------------- Result 144 ---------------------------------------------
[[1 (73%)]] --> [[[FAILED]]]

this movie may be the worst movie i have ever seen  basically it is right above leprachaun    the only difference is that it missing ice t  the scene where he does the chick with the carrott priceless  oh yeah they made a second one  genius




[Succeeded / Failed / Skipped / Total] 79 / 58 / 8 / 145:   7%|▋         | 145/2000 [22:54<4:52:58,  9.48s/it]

--------------------------------------------- Result 145 ---------------------------------------------
[[0 (73%)]] --> [[[FAILED]]]

this movie is well made  it is beautiful and wise  it is heart warming  it is great  and again it shows how great peter falk is  he is fantastic and he even gets better  the older he gets  thank you  peter falk  thank you very much for this gem of a movie       this movie entertains  there is lot of wisdom in this movie  there is lot of humor in this movie  there is life in this movie  and meaning  this movie shows  how life can be     peter falk is in that movie  he is just great  where is the oscar for peter falk  he deserves it so very much     peter falk just turned    i do sincerely hope that there will be more movies     walter j  langbein




#### MT BERT

In [ ]:
attack_model(bert_teacher, 
             dataset_for_attack,
             data_name.value,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value)

# Creating one log file dataset

In [ ]:
import os 
import pandas as pd
os.chdir('/content/drive/MyDrive/Master Thesis/Mean-Teacher-BERT-Robustness-Assessment')

# Creating one Dataset file by reading experiment gsheet
# link to fake news codalab dataset, saving my experiment result in  here
fakenews_link= 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTa7adpGv5lAg1gJD6lvUhhXdibgWpimrX8Lu7Ht-_nsKXCVd4gAyNA2krNE6dEtNEtc067FF9jxdR7/pub?gid=923968876&single=true&output=csv'

# link to imdb sentiment analysis review 
imdb_link= 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTa7adpGv5lAg1gJD6lvUhhXdibgWpimrX8Lu7Ht-_nsKXCVd4gAyNA2krNE6dEtNEtc067FF9jxdR7/pub?gid=235103714&single=true&output=csv'

# for custom sortings
custom_dict = {'BERT': 0, 'MTBERT': 1, 'DistilBERT': 3, 'MTDistilBERT':4} 

#reading fakenews data codalab 
df_fakenews= pd.read_csv(fakenews_link,
                          usecols=['Model',
                          'Attack_Recipe',
                          'Total_examples',
                          'Original_Accuracy_%',
                          'Accuracy_Under_Attack_%',
                          'Attack_success_rate_%',
                          'Average_perturbed_word_%',
                          'Avg_num_queries']).round(2).reset_index(drop=True)

#reading IMDB sentiment analysis dataset
df_imdb= pd.read_csv(imdb_link,
                          usecols=['Model',
                          'Attack_Recipe',
                          'Total_examples',
                          'Original_Accuracy_%',
                          'Accuracy_Under_Attack_%',
                          'Attack_success_rate_%',
                          'Average_perturbed_word_%',
                          'Avg_num_queries']).round(2).reset_index(drop=True)
# data looks like  
df_imdb.head()

#creating one complete file
df_fakenews['Dataset']='FakeNews'
df_imdb['Dataset']='IMDB'
df_imdb_fake_comp= df_fakenews.append(df_imdb)

#saving to folder 
save_loc='Result/Complete/complete_exp_result.csv'
df_imdb_fake_comp.to_csv(save_loc, index=False)


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'complete_exp_result'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df) 

In [ ]:
## Creating one report log file
# joining the result reports ,
# this one for google cloud report
dataset_name=['IMDB','codalab']
attack_recipes=['TextFoolerJin2019', 'BAEGarg2019', 'TextBuggerLi2018', 'PWWSRen2019']
for dataset in dataset_name:
    df= pd.DataFrame()
    for atk_rec in attack_recipes:
        file_list=os.listdir(f'Result/{dataset}/{atk_rec}')
        for f in file_list:
            df_temp= pd.read_csv(f'Result/{dataset}/{atk_rec}/{f}')
            df_temp['model']=f.split('_')[0]
            df_temp['attack_recipe']= atk_rec
            df= df.append(df_temp)

    if dataset=='codalab':
      df['Dataset']='FakeNews'
    else:
      df['Dataset']='IMDB'

    df.to_csv(f'Result/Complete/log_result_complete.csv',index=False)

In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'log_result_complete'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df) 